In [ ]:
# !jupyter nbextension enable --py widgetsnbextension

# Import Libraries and Modzy SDK

In [2]:
from modzy import ApiClient
from bs4 import BeautifulSoup
import requests
import json
import IPython
from api_key import API_KEY

# Web Scraping CoinDesk webpage 

In [3]:
# web scraping article from CoinDesk webpage
url = "https://www.coindesk.com/markets/2021/11/30/market-wrap-bitcoin-underperforms-as-ether-and-other-altcoins-rise/" 
# "https://www.coindesk.com/markets/2021/11/29/market-wrap-bitcoin-bears-retreat-as-traders-buy-on-dips/" 
# "https://www.coindesk.com/tech/2021/11/30/ethereum-in-2022-what-is-money-in-the-metaverse/" 

r = requests.get(url)
r.status_code

200

In [6]:
soup = BeautifulSoup(r.content, features='html.parser')

title = soup.find('div', class_='at-headline').text
paragraphs = soup.findAll('div',class_='at-text')
article = ''.join([p.text for p in paragraphs])

print(f"title: \n{title}")
print()
print(f"article: \n{article}")

title: 
Market Wrap: Bitcoin Underperforms as Ether and Other Altcoins Rise

article: 
Bitcoin was roughly flat on Tuesday while alternative cryptocurrencies such as ether and Solana’s SOL token were up about 5% over the past 24 hours. LUNA, the native token of the Terra blockchain, rallied about 13% as traders chased incentive programs.Overall, trading conditions were choppy across global markets after the U.S. Federal Reserve Chair Jerome Powell suggested that monetary policy could tighten faster than expected – potentially a negative for speculative assets, including cryptocurrencies and equities.Despite short-term price swings, some analysts remain bullish on bitcoin.“As BTC is looking good to close November below the expected target of $60,000, investors are optimistic that the cryptocurrency will repeat its historic trend of ending the year on a stellar bullish note,” Nikita Rudenia, co-founder of asset management firm 8848 Invest, wrote in an email to CoinDesk. Rudenia has a $70

# Initialize Modzy API Client

In [ ]:
# create modzy client
client = ApiClient(base_url="https://app.modzy.com/api", api_key=API_KEY)

# Submit Text Summarization Job

In [ ]:
sources = {}

# Add any number of inputs
sources["input_article"] = {
    "input.txt": article,
}

# submit the job
print("running text summarization job...")
job = client.jobs.submit_text("rs2qqwbjwb", "0.0.2", sources)
print(f"job: {job}")

# result = client.results.block_until_complete(job, timeout=None)
# print("running text summarization job...DONE")

In [ ]:
result = client.results.get(job.job_identifier)
result

In [ ]:
summary = result.get_first_outputs()['results.json']['summary']
summary

# Submit Text to Speech Job


In [ ]:
sources = {}

# sample_text = 'Analysts view the rotation from Bitcoin to altcoins as an indicator of greater appetite for risk.'

#Add any number of inputs
sources["summary"] = {
    "input.txt": summary,
}

# submit the job
print("running text-to-speech job...")
job = client.jobs.submit_text("uvdncymn6q", "0.0.2", sources)
print(f"job: {job}")


# speech = client.results.block_until_complete(job, timeout=None)
# print("running text-to-speech job...DONE")

In [ ]:
result = client.results.get(job.job_identifier)
result

In [ ]:
# get result .wav file from modzy
wav_url = speech.get_first_outputs()['results.wav']

headers = {
    'Accept': 'application/json', 
    'Authorization': f'ApiKey {API_KEY}'
}

wav =  requests.get(wav_url, headers=headers)
wav

# Play audio summary

In [ ]:
# play audio
IPython.display.Audio(wav.content)

In [ ]:
# save audio file            
with open('audio.wav', 'wb') as f:
     f.write(wav.content)

In [ ]:
# with requests.get(wav_url, headers=headers, auth=None, stream=True) as r:
#     with open('audio.wav', 'wb') as f:
#         for chunk in r.iter_content(chunk_size=8192):
#             f.write(chunk)

# IPython.display.display(IPython.display.Audio('audio.wav'))
# IPython.display.display(IPython.display.Audio(r.content))

# with open('market.txt', 'r', encoding='utf8') as file:
#     input_txt = file.read().replace('\n', '')
# print(input_txt)